## Precarga de librerias y funciones

In [11]:
%run "../recurrentes.ipynb"

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.



The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Note: you may need to restart the kernel to use updated packages.


In [12]:
%run "../funciones.ipynb"

#### Excel

In [4]:
# Activar Excel de seguimiento de medidas? 

excel = True
if excel == True:
    # Ingresar path excel para anotar resultados
    path_excel = base_path_l + r'\resultados_backtesting.xlsx'
    date = datetime.now().strftime("%d-%m-%Y %H-%M-%S")
    

In [5]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l
# d) dataset_10_meses_l

dataset = dataset_10_meses_l
dataset_name = os.path.basename(dataset)
df_train = pd.read_parquet(dataset)


## Formateo pre modelo

In [9]:

# [202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106] modelo 10 meses

ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = [202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106]
mes_test = 202106

In [7]:
df_train['foto_mes'].unique()

array([202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106,
       202107, 202108], dtype=int64)

In [10]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)
    

data = df_train

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

# Especificar mes de train y test


df_train = data[data['foto_mes'].isin(mes_train)]
df_test = data[data['foto_mes']==mes_test]

clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']

w_train = df_train['clase_peso']
w_test = df_test['clase_peso']

## Cross-Validation

In [ ]:
# Ordenar entradas en df_train en base a la columna foto_mes de meses más antiguos a más recientes
X_train = X_train.sort_values('foto_mes', ascending=True)

In [ ]:
semillas

[400009, 500009, 500011, 500021, 600009]

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)

lista_cv = []

params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=s,
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final =  max_gan * 5
lista_cv.append(final)



[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.071578 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.134004 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhea

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)


params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[1],
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final_1 =  max_gan * 5
lista_cv.append(final_1)

[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.089561 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.130294 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhea

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)


params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[2],
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final_2 =  max_gan * 5
lista_cv.append(final_2)

[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.062586 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.129290 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhea

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)




params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[3],
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final_3 =  max_gan * 5
lista_cv.append(final_3)

[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.074897 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.156746 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhea

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)



params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[4],
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final_4 =  max_gan * 5
lista_cv.append(final_4)

[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.074043 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.440371 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.191630 seconds.
You can set `force_row_wise=t

In [ ]:
finalisimo =  sum(lista_cv)/ len(lista_cv)  
finalisimo

10737406704.0

In [ ]:
# RESULTADOS:
# 1. Modelo Base = 10737406704.0

## Predicción y scoring

In [ ]:
# data = pd.read_parquet(dataset)

In [ ]:
# data_frame_meses = pd.DataFrame({'mes':[] , 'ganancia':[] })

In [ ]:
# data_frame_meses.sort_values('ganancia', ascending=False)

,mes,ganancia


In [ ]:
# meses_train = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
#        201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
#        202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
#        202101, 202102, 202103, 202104]

In [ ]:
# data.shape

(4735593, 157)

In [ ]:
# for mes in meses_train:
#     mes_train = mes
#     mes_test = 202106
        
#     data['clase_peso'] = 1.0
#     data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
#     data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
#     data['clase_peso'].isna().sum()
    
#     data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)
#     # Especificar mes de train y test

#     df_train = data[data['foto_mes']>=mes_train]
#     df_test = data[data['foto_mes']==mes_test]
    
#     clase_peso = df_train['clase_peso']
#     X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
#     Y_train = df_train['clase_binaria']
#     X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
#     Y_test = df_test['clase_binaria']
#     w_train = df_train['clase_peso']
#     w_test = df_test['clase_peso']
    
    
#     dicc_resultados = calcular_medidas(X_train, Y_train, w_train, X_test, Y_test, w_test)
#     meses = df_train['foto_mes'].unique().tolist()
    
#     ganancia = dicc_resultados['ganancia']
#     nueva_fila = pd.DataFrame({'mes':[meses], 'ganancia':[ganancia]})
#     data_frame_meses = pd.concat ( [data_frame_meses, nueva_fila], ignore_index=True)

ValueError: DataFrame for label cannot have multiple columns

<!-- #### Medidas -->

In [15]:
dicc_medidas = calcular_medidas(X_train, Y_train, w_train, X_test, Y_test, w_test)

[LightGBM] [Info] Number of positive: 333766, number of negative: 970448
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.219239 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 49926
[LightGBM] [Info] Number of data points in the train set: 1304214, number of used features: 295
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.255916 -> initscore=-1.067303
[LightGBM] [Info] Start training from score -1.067303
[LightGBM] [Info] Number of positive: 333766, number of negative: 970448
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.803860 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 49926
[LightGBM] [Info] Number of data points in the train set: 1304214, number of used features: 295
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.255916 -> initscore=

In [16]:
if excel == True:
    dicc_medidas['fecha'] = date
    dicc_medidas['dataset'] = dataset_name
    

In [17]:
cantidad_columnas = df_train.shape[1]
mes_train = df_train['foto_mes'].max()
mes_test = df_test['foto_mes'].max()
# Agregar nota
consideraciones = 'entrenando sólo noviembre'

dicc_medidas['cantidad_columnas'] = cantidad_columnas
dicc_medidas['mes_train'] = mes_train
dicc_medidas['mes_test'] = mes_test
dicc_medidas['consideraciones'] = consideraciones

In [18]:
dicc_medidas 

{'accuracy_score': 0.9144144690555327,
 'precision_score': 0.10829671749806151,
 'recall_score': 0.8434826371414192,
 'f1_score': 0.19194869151921207,
 'roc_auc_score': 0.8793811837549762,
 'ganancia': 142786000,
 'fecha': '24-11-2024 17-17-52',
 'dataset': 'dataset_10_meses.parquet',
 'cantidad_columnas': 298,
 'mes_train': 202106,
 'mes_test': 202106,
 'consideraciones': 'entrenando sólo noviembre'}

In [20]:
if excel == True:
    anotar_excel(path_excel, dicc_medidas)


Primera celda vacía en la fila 21
Fila a actualizar: 21
se actualizó fila 21 en el excel
